In [164]:
DF = r'../data/processed/clusters_skills_df.pkl'
CLUSTERS_SKILLS = r'..\data\processed\clusters_skills.yaml'
MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "rf_job_predict"
EXPERIMENT_ID = 'c410d8ab2d42489ba0b8738b000d13e5'
ARTIFACT_PATH = '../models/MODELS'

In [165]:
import mlflow
import mlflow.sklearn
import os
import pickle
import yaml
import pandas as pd
import numpy as np

In [166]:
df = pd.read_pickle(DF)

In [173]:
x = df.droplevel(0, axis=1)x

In [180]:
x =x.iloc[:,25:]

In [181]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [182]:
target_path = ARTIFACT_PATH + '/' + EXPERIMENT_ID + '/artifacts'

In [183]:
target_path

'../models/MODELS/c410d8ab2d42489ba0b8738b000d13e5/artifacts'

In [184]:
# Get a list of folders in the specified path
folders = [entry.name for entry in os.scandir(target_path) if entry.is_dir()]

# Print the list of folders
print(folders[0])

 Basic RandomForestClassifier with PCA


In [185]:
target_path += '/' + folders[0]+ '/model.pkl'
target_path

'../models/MODELS/c410d8ab2d42489ba0b8738b000d13e5/artifacts/ Basic RandomForestClassifier with PCA/model.pkl'

In [186]:
with open(target_path, 'rb') as file:
    loaded_model = pickle.load(file)

In [187]:
loaded_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  Index(['skills_group_0', 'skills_group_1', 'skills_group_10',
       'skills_group_11', 'skills_group_12', 'skills_group_13',
       'skills_group_14', 'skills_group_15', 'skills_group_16',
       'skills_group_17', 'skills_group_18', 'skills_group_19',
       'skills_group_2', 'skills_group_20', 'skills_group_21',
       'skills_group_22', 'skills_group_23', 'skills_group_24',
       'skills_group_3', 'skills_group_4', 'skills_group_5', 'skills_group_6',
       'skills_group_7', 'skills_group_9'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.99)),
                ('classifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42,
                                        verbose=1))])

In [188]:
with open(CLUSTERS_SKILLS, 'r') as file :
    clusters_skills = yaml.load(file, Loader=yaml.FullLoader)
    

In [241]:
del clusters_skills['skills_group_8']

In [189]:
# clusters_skills = pd.Series(clusters_skills)

In [190]:
# clusters_skills = clusters_skills.reset_index()

In [191]:
# clusters_skills.columns = ['Cluster', 'Skills']

In [291]:
sample_skills =  ['Python', 'Pandas', 'Keras']


In [292]:
test = pd.DataFrame(columns=list(clusters_skills.keys()) + x.columns.to_list())

In [293]:
test.loc[0] = 0

In [294]:
for cluster, skills in clusters_skills.items() :
    for skill in skills :
        if skill in sample_skills:
            test[cluster] +=1

In [295]:
for skill in sample_skills :
    if skill in test.columns.tolist() :
        test[skill] += 1

In [296]:
test.sum()

skills_group_0     0
skills_group_1     0
skills_group_10    0
skills_group_11    0
skills_group_12    0
                  ..
liblittletest      0
npm                0
pnpm               0
snitch             0
tunit              0
Length: 260, dtype: int64

In [297]:
class_names = loaded_model.classes_
class_names

array(['Academic researcher', 'Cloud infrastructure engineer',
       'Data or business analyst',
       'Data scientist or machine learning specialist',
       'Database administrator', 'DevOps specialist',
       'Developer, QA or test', 'Developer, back-end',
       'Developer, desktop or enterprise applications',
       'Developer, front-end', 'Developer, full-stack',
       'Developer, game or graphics', 'Developer, mobile',
       'Engineer, data', 'Scientist', 'Security professional',
       'System administrator'], dtype=object)

In [300]:
predictions = loaded_model.predict_proba(test)


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


array([[0.42543669, 0.        , 0.1359818 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.43858151,
        0.        , 0.        ]])

In [301]:
for class_name, probability in zip(class_names, predictions[0]):  # Assuming predictions has shape (1, n_classes)
    print(f"Class: {class_name}, Probability: {probability:.4f}")

Class: Academic researcher, Probability: 0.4254
Class: Cloud infrastructure engineer, Probability: 0.0000
Class: Data or business analyst, Probability: 0.1360
Class: Data scientist or machine learning specialist, Probability: 0.0000
Class: Database administrator, Probability: 0.0000
Class: DevOps specialist, Probability: 0.0000
Class: Developer, QA or test, Probability: 0.0000
Class: Developer, back-end, Probability: 0.0000
Class: Developer, desktop or enterprise applications, Probability: 0.0000
Class: Developer, front-end, Probability: 0.0000
Class: Developer, full-stack, Probability: 0.0000
Class: Developer, game or graphics, Probability: 0.0000
Class: Developer, mobile, Probability: 0.0000
Class: Engineer, data, Probability: 0.0000
Class: Scientist, Probability: 0.4386
Class: Security professional, Probability: 0.0000
Class: System administrator, Probability: 0.0000


In [303]:
sorted_classes = sorted(zip(class_names, predictions[0]), key=lambda x: x[1], reverse=True)
for class_name, probability in sorted_classes:
    print(f"Class: {class_name}, {probability:.4f}")

Class: Scientist, 0.4386
Class: Academic researcher, 0.4254
Class: Data or business analyst, 0.1360
Class: Cloud infrastructure engineer, 0.0000
Class: Data scientist or machine learning specialist, 0.0000
Class: Database administrator, 0.0000
Class: DevOps specialist, 0.0000
Class: Developer, QA or test, 0.0000
Class: Developer, back-end, 0.0000
Class: Developer, desktop or enterprise applications, 0.0000
Class: Developer, front-end, 0.0000
Class: Developer, full-stack, 0.0000
Class: Developer, game or graphics, 0.0000
Class: Developer, mobile, 0.0000
Class: Engineer, data, 0.0000
Class: Security professional, 0.0000
Class: System administrator, 0.0000


In [304]:
idx = np.argmax(predictions)
class_names[idx]

'Scientist'

In [305]:
others = []
for skill in x.columns.tolist() :
    if skill not in sample_skills:
        others.append(skill)

In [319]:
target_class = "Engineer, data"
for skill in others :
    new = test.copy()
    for cluster, skills in clusters_skills.items() :
        if skill in skills:
            new[cluster] +=1
    predictions = loaded_model.predict_proba(new)
    target_class_index = list(class_names).index(target_class)
    target_class_score = predictions[0][target_class_index]
    d[skill] = target_class_score
    

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurr

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurr

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurr

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parall

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parall

In [320]:
sorted_items = sorted(d.items(), key=lambda x: x[1], reverse=True)


In [321]:
for key, value in sorted_items:
    print(f"{key}: {value}")

PowerShell: 0.36602813852813854
VBA: 0.36602813852813854
Visual Basic (.Net): 0.36602813852813854
Cosmos DB: 0.36602813852813854
Microsoft Access: 0.36602813852813854
Microsoft Azure: 0.36602813852813854
.NET (5+) : 0.36602813852813854
.NET MAUI: 0.36602813852813854
C#: 0.3660281385281385
F#: 0.3660281385281385
Microsoft SQL Server: 0.3660281385281385
VMware: 0.3660281385281385
ASP.NET: 0.3660281385281385
ASP.NET CORE: 0.3660281385281385
Blazor: 0.3660281385281385
.NET Framework (1.0 - 4.8): 0.3660281385281385
Xamarin: 0.3660281385281385
Chocolatey: 0.3660281385281385
MSBuild: 0.3660281385281385
NuGet: 0.3660281385281385
Visual Studio Solution: 0.3660281385281385
Scala: 0.16
Cassandra: 0.16
Snowflake: 0.16
Play Framework: 0.16
Apache Kafka: 0.16
Apache Spark: 0.16
Hadoop: 0.16
R: 0.15963419397939518
Django: 0.15963419397939518
FastAPI: 0.15963419397939518
Flask: 0.15963419397939518
CUDA: 0.15963419397939518
Hugging Face Transformers: 0.15963419397939518
NumPy: 0.15963419397939518
Openc